In [2]:
# Here are some imports that are used along this notebook
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from time import time
import tensorflow as tf
from collections import OrderedDict
import keras
from sklearn.model_selection import KFold
from keras.utils import np_utils
%matplotlib inline
gt0 = time()

Using TensorFlow backend.


In [3]:
train_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTrain+.txt"
test_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTest+.txt"

In [4]:
col_names = np.array(["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels"])

In [5]:
nominal_inx = [2, 3, 4]
binary_inx = [7, 12, 14, 15, 21, 22]
numeric_inx = [1, 5, 6, 8, 9, 10, 11, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]

In [6]:
nominal_cols = []
binary_cols = []
numeric_cols = []
nominal_cols.append(col_names[nominal_inx])
binary_cols.append(col_names[binary_inx])
numeric_cols.append(col_names[numeric_inx])
#print(nominal_cols)

In [7]:
attack_dict_five_class = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}

attack_two_class = []
for key in attack_dict_five_class.keys():
    if key == 'normal':
        pass
    else:
        attack_two_class.append(key)

In [11]:
#Load test and train data
train_df = pd.read_csv(train_nsl_kdd_dataset_path, names = col_names)
test_df  = pd.read_csv(test_nsl_kdd_dataset_path , names = col_names)
train_labels = train_df.pop('dst_host_srv_rerror_rate')
test_labels = test_df.pop('dst_host_srv_rerror_rate')
total_dataset = pd.concat([train_df, test_df])
#print(total_dataset.shape)


TypeError: drop() takes from 2 to 6 positional arguments but 10 were given

In [9]:

total_dataset.head()
total_dataset.shape
#train_df

total_dataset = total_dataset[['protocol_type','flag','count']]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,labels
0,tcp,ftp_data,SF,491,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20
0,udp,other,SF,146,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15
0,tcp,private,S0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19
0,tcp,http,SF,232,8153,0,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21
0,tcp,http,SF,199,420,0,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21


In [9]:
#print(pd.get_dummies(train_df, columns=['flag']))
#get = train_df.pop('protocol_type')
#print(get)
#pd.get_dummies(get)

In [10]:
#test_df.head(),train_df.head()
    

In [11]:
total_dataset = pd.get_dummies(total_dataset)
#print(total_dataset.shape)
#train_size = int(len(total_dataset) * 0.80)
#print(train_size)
#test_size = len(total_dataset) - train_size
#print(test_size)
train_df = total_dataset.iloc[0:125973, :]
test_df = total_dataset.iloc[125973:, :]
print(train_df.shape, test_df.shape)

(125973, 72) (22544, 72)


In [12]:
#total_dataset.head()

In [13]:
train_labels_for_two_class = pd.DataFrame(train_labels.as_matrix(), columns=["class"])
#print(train_labels_for_two_class)
test_labels_for_two_class = pd.DataFrame(test_labels.as_matrix(), columns=["class"])

In [14]:
train_labels_for_two_class.loc[train_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
train_labels_for_two_class.loc[train_labels_for_two_class['class'] == "normal" , 'class'] = 0
#print(train_labels_for_two_class)
#for test labels 
test_labels_for_two_class.loc[test_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
test_labels_for_two_class.loc[test_labels_for_two_class['class'] == "normal" , 'class'] = 0




In [15]:
train_labels_for_two_class = np_utils.to_categorical(train_labels_for_two_class)
print(train_labels_for_two_class.shape)
test_labels_for_two_class = np_utils.to_categorical(test_labels_for_two_class)
print(train_labels_for_two_class.shape)

(125973, 2)
(125973, 2)


In [16]:
#min-max mazimazation for the data
from sklearn.preprocessing import MinMaxScaler
train_X = train_df.as_matrix()
train_Y = train_labels_for_two_class

test_X = test_df.as_matrix()
test_Y = test_labels_for_two_class
scaler = MinMaxScaler() 
scaler.fit_transform(train_X)
scaler.fit_transform(train_Y)
scaler.fit_transform(test_X)
scaler.fit_transform(test_Y)
#train_X.shape[1]
    

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [17]:
#Training our auto encoder

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers



In [18]:
input_dim = train_X.shape[1]
input_layer = Input(shape=(input_dim, ))
encoder = Dense(30, activation="sigmoid", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
decoder = Dense(input_dim, activation='relu')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)


In [19]:
nb_epoch = 2
batch_size = 128
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)

history = autoencoder.fit(train_X,train_X ,
                    epochs=nb_epoch,
                    verbose = 1,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_X, test_X)).history

Train on 125973 samples, validate on 22544 samples
Epoch 1/2
125973/125973 [==============================] - 3s 21us/step - loss: 478647098046.6931 - acc: 0.5043 - val_loss: 3105903205.6707 - val_acc: 0.3975
Epoch 2/2
125973/125973 [==============================] - 3s 21us/step - loss: 478647094650.5942 - acc: 0.4918 - val_loss: 3105902763.4039 - val_acc: 0.3975


In [27]:
#  feat1 = Model(autoencoder, history)
#  print(encoder.input)
# X_train1 = autoencoder.predict(train_X)
# X_train1

encoder = Model(input_layer, encoder)

In [28]:
out2 = Dense(2, activation='softmax')(encoder.output)
newmodel = Model(encoder.input,out2)

In [29]:
test_X.shape

(22544, 72)

In [30]:
from keras import optimizers
opt = optimizers.SGD(lr=0.00001)
newmodel.compile(loss='categorical_crossentropy',
          optimizer=opt, 
          metrics=['accuracy']) 

newmodel.fit(train_X, train_Y,
      epochs=10,
      batch_size=128,
      shuffle=True,
      validation_data=(test_X, test_Y))


Train on 125973 samples, validate on 22544 samples
Epoch 1/10
125973/125973 [==============================] - 3s 26us/step - loss: 0.8354 - acc: 0.5346 - val_loss: 0.9934 - val_acc: 0.4308
Epoch 2/10
125973/125973 [==============================] - 3s 21us/step - loss: 0.8214 - acc: 0.5348 - val_loss: 0.9841 - val_acc: 0.4310
Epoch 3/10
125973/125973 [==============================] - 3s 21us/step - loss: 0.8082 - acc: 0.5366 - val_loss: 0.9755 - val_acc: 0.4314
Epoch 4/10
125973/125973 [==============================] - 2s 19us/step - loss: 0.7956 - acc: 0.5527 - val_loss: 0.9676 - val_acc: 0.4537
Epoch 5/10
125973/125973 [==============================] - 2s 16us/step - loss: 0.7834 - acc: 0.5712 - val_loss: 0.9601 - val_acc: 0.4696
Epoch 6/10
125973/125973 [==============================] - 2s 16us/step - loss: 0.7706 - acc: 0.5995 - val_loss: 0.9520 - val_acc: 0.4829
Epoch 7/10
125973/125973 [==============================] - 2s 16us/step - loss: 0.7527 - acc: 0.6342 - val_loss: 0

In [32]:
scores = newmodel.evaluate(test_X, test_Y, verbose=1, steps=50) 
#print("Accuracy: ", scores[1])
scores 

50/50 [==============================] - 1s 28ms/step


[35.076221466064453, 0.52639281749725342]